<a href="https://colab.research.google.com/github/atlmapguy/TSP_UoL/blob/master/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pwd

'/content'

In [2]:
pip install CuPy

  Using cached cupy-13.1.0.tar.gz (3.5 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for CuPy
  Running setup.py clean for CuPy
Failed to build CuPy
ERROR: Could not build wheels for CuPy, which is required to install pyproject.toml-based projects


In [3]:
pip install PyPDF2

In [4]:
pip install tqdm

In [5]:
!pip install spacy-transformers

In [6]:
import os
import PyPDF2

source_folder = '/content/NER_apps/resume_dataset/pdf_format'
combined_txt_path = '/content/NER_apps/resume_dataset/pdf_format/combined_resumes.txt'

# Create a list to store text from all PDF files
all_pdf_text = []

# Loop through PDF files in the source folder
for filename in os.listdir(source_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(source_folder, filename)

        # Extract text from PDF
        with open(pdf_path, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            pdf_text = ' '.join(page.extract_text() for page in pdf_reader.pages)
            all_pdf_text.append(pdf_text)

# Combine all PDF text into a single string
combined_text = '\n\n'.join(all_pdf_text)  # Adding double newline separator

# Save combined text to a single TXT file
with open(combined_txt_path, 'w', encoding='utf-8') as combined_txt_file:
    combined_txt_file.write(combined_text)

In [7]:
pip install -U spacy

In [8]:
pip install spacy-transformers

In [9]:
import json

# Load the annotated data from a JSON file
cv_data = json.load(open('/content/NER_apps/annotations_dataset/annotations.json','r'))

# Display the number of items in the dataset
len(cv_data)

# Display the first item in the dataset
cv_data[0]

["Kevin White \r\n587 53rd Ave. N.  \r\nSaint Petersburg, FL 33703 atlmapguy@gmail.com  \r\n(404) 274-3150 (Cell) \r\n \r\nQualifications Summary:   \r\nExecutive level leader, data scientist, and GIS (Geographic Information Systems) expert with proven analytical skills that support decisions in business and government.   \r\n \r\nSkills:  \r\n•\tPython  \r\n•\tR  \r\n•\tJupyter Notebook  \r\n•\tMicrosoft Azure Databricks\r\n•\tDataRobot  \r\n•\tAmazon Web Services (AWS)  SageMaker\r\n•\tDatabricks\r\n•\tDocker Desktop\r\n•\tTableau\r\n•\tSQL/noSQL\r\n•\tContinuous Integration / Continuous Development (CI/CD)\r\n•\tAgile for Data Science Teams\r\n•\tFlask\r\n•\tReact\t•\tESRI ArcGIS  \r\n•\tIBM SPSS  \r\n•\tStatistical Data Analysis  \r\n•\tData Visualization  \r\n•\teCommerce  \r\n•\tBig Data  \r\n•\tPredictive Modeling \r\n•\tSpark/PySpark/ Apache Spark\r\n•\tPower BI\r\n•\tAPIs (Application Programming Interfaces)\r\n•\tNatural Language Processing\r\n \r\nWork Experience:   \r\n07/2

In [10]:
# Initialize spaCy configuration files by copying from base_config to config.cfg
# !python -m spacy init fill-config /"your_file_path"/base_config.cfg /"your_file_path"/config/config.cfg
!python -m spacy init fill-config /content/NER_apps/config/base_config.cfg /content/NER_apps/config/config.cfg

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
/content/NER_apps/config/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [11]:
# Define a function to create spaCy DocBin objects from the annotated data
def get_spacy_doc(file, data):
  # Create a blank spaCy pipeline
  nlp = spacy.blank('en')
  db = DocBin()

  # Iterate through the data
  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    # Extract entities from the annotations
    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity:
        continue

      entity_indices = entity_indices + list(range(start, end))
      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        # Log errors for annotations that couldn't be processed
        err_data = str([start, end]) + "    " + str(text) + "\n"
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db

In [12]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import cupy; import cupyx

# Increase the size of the cv_data variable
cv_data1=cv_data*100

# Split the annotated data into training and testing sets
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data1, test_size=0.2)

# Display the number of items in the training and testing sets
len(train), len(test)

# Open a file to log errors during annotation processing
file = open('/content/NER_apps/trained_models/train_file.txt','w')

# Create spaCy DocBin objects for training and testing data
db = get_spacy_doc(file, train)
db.to_disk('/content/NER_apps/trained_models/train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('/content/NER_apps/trained_models/test_data.spacy')

# Close the error log file
file.close()

ImportError: 
================================================================
Failed to import CuPy.

If you installed CuPy via wheels (cupy-cudaXXX or cupy-rocm-X-X), make sure that the package matches with the version of CUDA or ROCm installed.

On Linux, you may need to set LD_LIBRARY_PATH environment variable depending on how you installed CUDA/ROCm.
On Windows, try setting CUDA_PATH environment variable.

Check the Installation Guide for details:
  https://docs.cupy.dev/en/latest/install.html

Original error:
  ImportError: libcuda.so.1: cannot open shared object file: No such file or directory
================================================================


In [ ]:
# Train a spaCy NER model using the provided configuration and data
!python -m spacy train /content/NER_apps/config/config.cfg  --output /content/NER_apps/trained_models/output --paths.dev /content/NER_apps/trained_models/test_data.spacy --gpu-id 0

# New Section